In [1]:
# !wget https://huggingface.co/mesolitica/conformer-medium-malay-whisper/resolve/main/malay-stt.model
# !wget https://gist.githubusercontent.com/huseinzol05/98974ae8c6c7a65d4bc0af9f5003786a/raw/2e06e71ef7349a57bc58cc9913ae6bae1f9f8447/mp.py

In [2]:
from glob import glob
import json
import soundfile as sf
from tqdm import tqdm
import mp
import os
import sentencepiece as spm
import malaya_speech
import numpy as np

sp_model = spm.SentencePieceProcessor(model_file='malay-stt.model')
sr = 16000

/home/husein/.local/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.2.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
/home/husein/.local/lib/python3.8/site-packages/whisper/timing.py:58: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def backtrace(trace: np.ndarray):
`pyaudio` is not available, `malaya_speech.streaming.pyaudio` is not able to use.


In [3]:
filtered = []

with open('pseudolabel.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        f = l['audio_filename']
        if not os.path.exists(f):
            continue
        t = l['predict_ms'][41:-13].strip()
        filtered.append((f, t))
        
len(filtered)

1611536

In [4]:
filtered = [(no, f) for no, f in enumerate(filtered)]

In [6]:
# with open('force-alignment/0.json') as fopen:
#     data = json.load(fopen)

In [7]:
# split, temp = [], []
# diag = data['diag']
# for no, r in enumerate(data['subwords_alignment']):
#     if r['score'] >= 0.05 or diag[no] > 0.1:
#         temp.append(r)
    
#     else:
#         if len(temp):
#             split.append(temp)
#             temp = []
            
# if len(temp):
#     split.append(temp)
    
# selected = []
# for s in split:
#     start = s[0]['start']
#     stack = [s[0]]
#     for s_ in s[1:]:
#         stack.append(s_)
#         start = stack[0]['start']
#         end = s_['end'] + 0.3
#         if end - start >= 0.5:
#             seq = [s__['text'] for s__ in stack]
#             merged = sp_model.Decode(sp_model.PieceToId(seq))
#             selected.append((merged, start, end))
            
# if np.mean(diag) > 0.5:
#     seq = [s['text'] for s in data['subwords_alignment']]
#     merged = sp_model.Decode(sp_model.PieceToId(seq))
#     start = data['subwords_alignment'][0]['start']
#     end = data['subwords_alignment'][-1]['end'] + 0.3
#     selected.append((merged, start, end))

In [8]:
!mkdir processed-audio processed
# !rm processed-audio/* processed/*

mkdir: cannot create directory ‘processed-audio’: File exists
mkdir: cannot create directory ‘processed’: File exists


In [9]:
def loop(rows):
    rows, index = rows
    for row in tqdm(rows):
        i, row = row
        filename = os.path.join('force-alignment', f'{i}.json')
        if not os.path.exists(filename):
            continue
        try:
            with open(filename) as fopen:
                data = json.load(fopen)
                
            if 'fatihah' in data['p']:
                continue
                
            f_json = os.path.join('processed', f'{index}-{i}.json')
            if os.path.exists(f_json):
                continue
                
            y, sr = malaya_speech.load(row[0])
            
            split, temp = [], []
            diag = data['diag']
            for no, r in enumerate(data['subwords_alignment']):
                if r['score'] >= 0.05 or diag[no] > 0.1:
                    temp.append(r)

                else:
                    if len(temp):
                        split.append(temp)
                        temp = []

            if len(temp):
                split.append(temp)

            selected = []
            for s in split:
                start = s[0]['start']
                stack = [s[0]]
                for s_ in s[1:]:
                    stack.append(s_)
                    start = stack[0]['start']
                    end = s_['end'] + 0.3
                    if end - start >= 0.5:
                        seq = [s__['text'] for s__ in stack]
                        merged = sp_model.Decode(sp_model.PieceToId(seq))
                        selected.append((merged, start, end))

            if np.mean(diag) > 0.5:
                seq = [s['text'] for s in data['subwords_alignment']]
                merged = sp_model.Decode(sp_model.PieceToId(seq))
                start = data['subwords_alignment'][0]['start']
                end = data['subwords_alignment'][-1]['end'] + 0.3
                selected.append((merged, start, end))
            
            if not len(selected):
                continue
                
            filtered = []
            for no, s in enumerate(selected):
                f = os.path.join('processed-audio', f'{index}-{i}-{no}.mp3')
                start = int(s[1] * sr)
                end = int(s[2] * sr)
                sf.write(f, y[start: end], sr)
                filtered.append(
                    {
                        'text': s[0],
                        'start': s[1],
                        'end': s[2],
                        'audio_filename': f,
                        'original_audio_filename': row[0],
                    }
                )
                
            with open(f_json, 'w') as fopen:
                json.dump(filtered, fopen)
            
        except:
            pass

In [10]:
loop((filtered[:10], 0))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 1095.52it/s]


In [13]:
mp.multiprocessing(filtered, loop, cores = 5, returned = False)

100%|█████████████████████████████████████████████████████████████████████████████████████████| 322307/322307 [12:38:46<00:00,  7.08it/s]


In [20]:
processed = glob('processed/*.json')
len(processed)

1046340

In [21]:
with open(processed[-1]) as fopen:
    d = json.load(fopen)
d

[{'text': 'nampak',
  'start': 0.28084332579185517,
  'end': 0.6209638009049774,
  'audio_filename': 'processed-audio/3-1276358-0.mp3',
  'original_audio_filename': 'output-audio/1-39886-6.mp3'}]

In [16]:
import IPython.display as ipd

ipd.Audio('processed-audio/3-1276358-0.mp3')

In [17]:
!du -hs processed-audio

195G	processed-audio


In [19]:
# !~/7zz -v10g a processed-audio.7z processed-audio > /dev/null

In [22]:
with open('processed.jsonl', 'w') as fopen_l:
    for f in tqdm(processed):
        try:
            with open(f) as fopen:
                data = json.load(fopen)
            fopen_l.write(f'{json.dumps(data)}\n')
        except:
            pass

100%|████████████████████████████████████████████████████████████████████████████████████████| 1046340/1046340 [07:47<00:00, 2239.37it/s]


In [23]:
from huggingface_hub import HfApi
api = HfApi()

In [24]:
api.upload_file(
    path_or_fileobj='processed.jsonl',
    path_in_repo='processed.jsonl',
    repo_id='mesolitica/pseudostreaming-malaya-speech-stt',
    repo_type='dataset',
)

processed.jsonl:   0%|          | 0.00/2.62G [00:00<?, ?B/s]

'https://huggingface.co/datasets/mesolitica/pseudostreaming-malaya-speech-stt/blob/main/processed.jsonl'

In [ ]:
for f in glob('*7z*'):
    print(f)
    api.upload_file(
        path_or_fileobj=f,
        path_in_repo=f,
        repo_id='mesolitica/pseudostreaming-malaya-speech-stt',
        repo_type='dataset',
    )

processed-audio.7z.002


processed-audio.7z.002:   0%|          | 0.00/10.7G [00:00<?, ?B/s]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [30]:
files = glob('processed-audio/*.mp3')
len(files)

12550048

In [32]:
def loop(files):
    files, _ = files
    total = 0
    for f in tqdm(files):
        y, sr = malaya_speech.load(f)
        total += len(y) / sr
    return [total]

In [39]:
totals = mp.multiprocessing(files, loop)

 41%|████████████████████████████████████▍                                                     | 848053/2091674 [15:36<24:07, 858.93it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 55%|████████████████████████████████████████████████▍                                       | 1151062/2091674 [21:11<15:31, 1009.71it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 74%|█████████████████████████████████████████████████████████████████▍     

In [40]:
sum(totals) / 60 / 60

8667.802379812754